In [1]:
import numpy as np 
import pandas as pd 

In [2]:
real_data = pd.read_csv('True.csv')
fake_data = pd.read_csv('Fake.csv')
snopes = pd.read_csv('snopes_phase1_clean_2018_7_3.csv')
articles= pd.read_csv('news_articles.csv')

In [3]:
real_data.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
real_data.count()

title      21417
text       21417
subject    21417
date       21417
dtype: int64

In [7]:
fake_data.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
fake_data.count()

title      23481
text       23481
subject    23481
date       23481
dtype: int64

In [11]:
fake_data['label']='Fake' #Creating a new column which is called label and value in it is Fake
real_data['label']='Real'#Creating a new column which is called label and value in it is Real
#This is necessary if we want to concatenate the dataframes since we need a way to identify which articles are fake and which real.

In [15]:
articles.head()

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0


In [17]:
articles['label'].value_counts()

Fake    1294
Real     801
Name: label, dtype: int64

In [20]:
articles.drop(['author','site_url','main_img_url','title_without_stopwords','hasImage','text_without_stopwords','language'],inplace=True,axis=1)

#Dropping unneccessary columns so we could concatenate it easier with our other two datasets

In [21]:
articles.isna().sum()#Checking how many NaN values exist

published     0
title         0
text         46
type          1
label         1
dtype: int64

In [22]:
articles = articles.dropna()#Dropping the rows with NaN values

In [23]:
articles.isna().sum()

published    0
title        0
text         0
type         0
label        0
dtype: int64

In [24]:

from dateutil.parser import parse
def is_valid_date(s): #Function to check whether date is valid or not
    try:
        parse(s)
        return True
    except:
        return False

articles = articles[articles.published.apply(is_valid_date)]#Leaving only the rows with valid dates.
articles['published'] = pd.to_datetime(articles['published'], utc=True)#Coverting argmunets in published column to datetime
articles['date']=articles['published'].dt.strftime("%d %B, %Y")#Creating a new column called date 
articles.drop('published',inplace=True,axis=1)

In [25]:
articles.rename(columns={'type': 'subject'}, inplace=True)#Changing the column name type to subject

In [26]:
columns = list(articles.columns)
columns[3],columns[4]=columns[4],columns[3] #Swapping the places of columns date and label
articles =articles[columns]

['title', 'text', 'subject', 'label', 'date']

In [29]:
articles.head()

,title,text,subject,date,label
0,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,bias,"26 October, 2016",Real
1,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,bias,"29 October, 2016",Real
2,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,bias,"30 October, 2016",Real
3,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,bias,"01 November, 2016",Real
4,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,bias,"01 November, 2016",Real


In [30]:
news = pd.concat([real_data,fake_data,articles]) #Putting the dataframes all together into our new dataframe called news

In [38]:
news.isna().sum()#Checking if there are any NaN values

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [31]:
news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",Real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",Real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",Real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",Real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",Real


In [32]:
news['label'].value_counts()

Fake    24772
Real    22174
Name: label, dtype: int64

Now we have put the data together from the first and second Kaggle dataset.

In [33]:
snopes.head()

,fact_rating_phase1,snopes_url_phase1,article_title_phase1,article_category_phase1,article_date_phase1,article_claim_phase1,article_origin_url_phase1,index_paragraph_phase1,page_is_first_citation_phase1
0,true,https://www.snopes.com/fact-check/lori-mcallen...,Did an Oregon Department of Transportation Emp...,Viral Phenomena Viral Phenomena,21 June 2018,"In June 2018, Oregon DOT employee Lori McAllen...",https://www.oregon.gov/odot/Pages/index.aspx,1,True
1,true,https://www.snopes.com/fact-check/lori-mcallen...,Did an Oregon Department of Transportation Emp...,Viral Phenomena Viral Phenomena,21 June 2018,"In June 2018, Oregon DOT employee Lori McAllen...",http://archive.is/HpML7,2,False
2,true,https://www.snopes.com/fact-check/lori-mcallen...,Did an Oregon Department of Transportation Emp...,Viral Phenomena Viral Phenomena,21 June 2018,"In June 2018, Oregon DOT employee Lori McAllen...",https://us-east-1.tchyn.io/snopes-production/u...,4,False
3,true,https://www.snopes.com/fact-check/lori-mcallen...,Did an Oregon Department of Transportation Emp...,Viral Phenomena Viral Phenomena,21 June 2018,"In June 2018, Oregon DOT employee Lori McAllen...",https://us-east-1.tchyn.io/snopes-production/u...,10,False
4,true,https://www.snopes.com/fact-check/lori-mcallen...,Did an Oregon Department of Transportation Emp...,Viral Phenomena Viral Phenomena,21 June 2018,"In June 2018, Oregon DOT employee Lori McAllen...",https://us-east-1.tchyn.io/snopes-production/u...,12,False


In [34]:
snopes.count()

fact_rating_phase1               37957
snopes_url_phase1                37957
article_title_phase1             37957
article_category_phase1          37950
article_date_phase1              37957
article_claim_phase1             37957
article_origin_url_phase1        37956
index_paragraph_phase1           37957
page_is_first_citation_phase1    37957
dtype: int64